In [2]:
import tensorflow as tf

In [3]:
#import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#define some parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

In [5]:
#where to save tensorboard model summaries
LOG_DIR = 'logs/RNN_with_summaries'

In [6]:
#create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32, shape = [None, time_steps, element_size], name = 'inputs')
y = tf.placeholder(tf.float32, shape = [None, num_classes], name = 'labels')

In [1]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [7]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
        
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
        
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def rnn_step(previous_hidden_state, x):
    
    current_hidden_state = tf.tanh(
        tf.matmul(previous_hidden_state, Wh) +
        tf.matmul(x, Wx) + b_rnn)
    
    return current_hidden_state

In [9]:
#current input shape: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm = [1, 0, 2])
#current input shape now: (time_steps, batch_size, element_size)

initial_hidden = tf.zeros([batch_size, hidden_layer_size])
all_hidden_states = tf.scan(rnn_step, 
                            processed_input,
                            initializer = initial_hidden,
                            name = 'states')

In [10]:
#weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope('W_linear'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size,
                                              num_classes],
                                              mean = 0, stddev = 0.01))
        variable_summaries(Wl)
        
    with tf.name_scope('Bias_linear'):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                              mean = 0, stddev = 0.01))
        variable_summaries(bl)

In [11]:
#apply linear layer to state vector
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

with tf.name_scope('linear_layer_weights') as scope:
    #iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    #get last output
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

In [12]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = y))
    tf.summary.scalar('cross_entropy', cross_entropy)
    
with tf.name_scope('train'):
    #using RMSPropOptimizer
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)
    
#merge all the summaries
merged = tf.summary.merge_all()
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
#get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

with tf.Session() as sess:
    #write summaries to LOG_DIR --used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train',
                                         graph = tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test',
                                        graph = tf.get_default_graph())
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #reshape data to get 28 sequences of 28 pixels
        batch_x = batch_x.reshape((batch_size, time_steps, element_size))
        summary, _ = sess.run([merged, train_step], 
                               feed_dict = {_inputs: batch_x,
                                                  y: batch_y})
        #add to summaries
        train_writer.add_summary(summary, i)
        
        if i % 1000 == 0:
            acc, loss, = sess.run([accuracy, cross_entropy],
                                   feed_dict = {_inputs: batch_x,
                                                      y: batch_y})
            print('Iter ' + str(i) + ', minibatch loss = ' + \
                  '{:.6f}'.format(loss) + ', training accuracy = ' + \
                  '{:.5f}'.format(acc))
            
        if i % 10:
            #add to summaries
            summary, acc = sess.run([merged, accuracy],
                                    feed_dict = {_inputs: test_data,
                                                       y: test_label})
            test_writer.add_summary(summary, i)
            
    test_acc = sess.run(accuracy, feed_dict = {_inputs: test_data,
                                                     y: test_label})
    print('Test accuracy: ', test_acc)

Iter 0, minibatch loss = 2.303271, training accuracy = 5.46875
Iter 1000, minibatch loss = 1.193217, training accuracy = 58.59375
Iter 2000, minibatch loss = 0.381518, training accuracy = 88.28125
Iter 3000, minibatch loss = 0.316441, training accuracy = 93.75000
Iter 4000, minibatch loss = 0.066740, training accuracy = 100.00000
Iter 5000, minibatch loss = 0.051956, training accuracy = 99.21875
Iter 6000, minibatch loss = 0.026636, training accuracy = 100.00000
Iter 7000, minibatch loss = 0.047005, training accuracy = 98.43750
Iter 8000, minibatch loss = 0.152049, training accuracy = 97.65625
Iter 9000, minibatch loss = 0.030562, training accuracy = 99.21875
Test accuracy:  99.21875
